In [1]:
# !pip install spacy==2.2.4
# compile neuralcoref from scratch according to https://github.com/huggingface/neuralcoref

In [2]:
import spacy
print(spacy.__version__) 

2.2.4


In [3]:
import en_core_web_lg
# Add neural coref to SpaCy's pipe
import neuralcoref

nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

In [9]:

doc = nlp(u'Yuling and Doruk will first work on coreferencing for their project. Then they will work on NER and Mari is totally okay with this! Can you believe it!?')


In [10]:

doc._.has_coref
doc._.coref_clusters

[Yuling and Doruk: [Yuling and Doruk, their, they], this: [this, it]]